In [1]:
import pandas as pd
import numpy as np
import preprocessor as pre
from datetime import datetime

# DATA

In [2]:
ds=pd.read_json('../dataset/Tweet-Fact-Not-Fact.jsonl')
ds.head()

,id,text,labels
0,591974898139987968,RT @Economynext: #SriLanka to fly emergency me...,1
1,591979323164233729,India committed in aid to Nepal: President Pra...,-1
10,591977175768698881,RT @iamamkay: need all out support @nytimes: N...,-1
100,591996410792321024,RT @MERCYMalaysia: MERCY Malaysia will be resp...,-1
1000,592204836038311937,RT @AmeriCares: #Nepal is a very poor country ...,-1


In [3]:
df=ds.loc[ds['labels']!=-1]
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
total_len=len(df)

In [5]:
total_len

1548

In [6]:
txt=''
for ix in range(total_len):
    pre.set_options(pre.OPT.URL, pre.OPT.EMOJI,pre.OPT.SMILEY,pre.OPT.MENTION,pre.OPT.HASHTAG)
    tweet=pre.clean(df.text[ix].replace("\n",""))
    line=f'__label__{df.labels[ix]} {tweet}\n'
    txt=txt+line
file=open('../../fastText/experiments/input/Tweets_labels.txt','w')
file.write(txt[:-1])
file.close()

# fast text

In [4]:
ds.head()

,id,text,labels
0,591974898139987968,RT @Economynext: #SriLanka to fly emergency me...,1
1,591979323164233729,India committed in aid to Nepal: President Pra...,-1
10,591977175768698881,RT @iamamkay: need all out support @nytimes: N...,-1
100,591996410792321024,RT @MERCYMalaysia: MERCY Malaysia will be resp...,-1
1000,592204836038311937,RT @AmeriCares: #Nepal is a very poor country ...,-1


In [5]:
from fastText import FastText
m=FastText.load_model('../../fastText/experiments/result/fire_1500model.bin')

In [29]:
ds['scores']=np.zeros((len(ds)))
for ix in range(len(ds)):
    pre.set_options(pre.OPT.URL, pre.OPT.EMOJI,pre.OPT.SMILEY,pre.OPT.MENTION,pre.OPT.HASHTAG)
    prediction=m.predict(pre.clean(ds.text[ix].replace('\n','')))
    if prediction[0][0]=='__label__1':
        sc=float(prediction[1][0])
        if sc>1.0:
            sc=1.0
        ds.loc[ix,'scores']=sc
    else:
        sc=float(1.0-float(prediction[1][0]))
        ds.loc[ix,'scores']=sc
#scores=np.array(scores)
#scores=np.minimum(scores,1.0)


In [4]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [8]:
id2tweet={}
id2vec={}
for ix in range(len(ds)):
    pre.set_options(pre.OPT.URL, pre.OPT.EMOJI,pre.OPT.SMILEY,pre.OPT.MENTION,pre.OPT.HASHTAG)
    tweet=pre.clean(ds.text[ix].replace("\n"," "))
    id2tweet[ds.id[ix]]=tweet
    id2vec[ds.id[ix]]=m.get_sentence_vector(tweet)
    

In [5]:
ds['pnouns']=np.zeros((len(ds)))
ds['numbers']=np.zeros((len(ds)))
ds['ent']=np.zeros((len(ds)))
for ix in range(len(ds)):
    if (ix%5000==0):
        print (ix)
    pre.set_options(pre.OPT.URL, pre.OPT.EMOJI,pre.OPT.SMILEY,pre.OPT.MENTION,pre.OPT.HASHTAG)
    sent=(pre.clean(ds.text[ix].replace('\n','')))
    doc=nlp(sent)
    pnoun=0
    num=0
    for token in doc:
        if (token.pos_ == "PROPN") :
            pnoun+=1
        if (token.pos_ == "NUM"):
            num+=1
    ds.loc[ix,'pnouns']=pnoun
    ds.loc[ix,'ent']=len(doc.ents)
    ds.loc[ix,'numbers']=num
    


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


In [7]:
ds.pnouns.max()

18.0

In [20]:
ds.head()

,id,text,labels,scores,pnouns,numbers,ent
0,591974898139987968,RT @Economynext: #SriLanka to fly emergency me...,1,0.999413,1.0,0.0,1.0
1,591979323164233729,India committed in aid to Nepal: President Pra...,-1,0.999755,9.0,0.0,5.0
10,591977175768698881,RT @iamamkay: need all out support @nytimes: N...,-1,0.000000,1.0,1.0,3.0
100,591996410792321024,RT @MERCYMalaysia: MERCY Malaysia will be resp...,-1,0.000000,4.0,0.0,2.0
1000,592204836038311937,RT @AmeriCares: #Nepal is a very poor country ...,-1,0.000000,1.0,0.0,2.0


In [30]:
ds_sorted=ds.sort_values('scores',ascending=False).reset_index(drop=True)

In [31]:
ds_sorted.head()

,id,text,labels,scores,pnouns,numbers,ent
0,592219443742134272,RT @RadioPakistan: Pakistan sends medical and ...,-1,1.0,3.0,0.0,2.0
1,594082185763749888,7.1 magnitude earthquake hits Papua New Guinea...,-1,1.0,9.0,1.0,3.0
2,594389980580958209,"Nepal quake: Death toll rises to 6,624: The de...",-1,1.0,1.0,2.0,4.0
3,595426274102870016,RT @firstpost: Strong earthquake of magnitude ...,-1,1.0,4.0,1.0,2.0
4,593106696131596288,"Nepal earthquake: Death toll crosses 5,000 mar...",-1,1.0,0.0,1.0,1.0


In [37]:
data_test=ds.loc[ds['labels']!=-1].reset_index(drop=True)

In [45]:
pnouns_max=ds.pnouns.max()
numbers_max=ds.numbers.max()
ent_max=ds.ent.max()

In [72]:
ds['combined']=np.zeros((len(ds)))
for ix in range(len(ds)):
    sc=ds.scores[ix]
    pn=ds.pnouns[ix]/(pnouns_max) 
    num=ds.numbers[ix]/(numbers_max)
    ent=ds.ent[ix]/ent_max
    a,b,c,d=0.7,0.1,0.1,0.1
    combined_score=a*sc+b*pn+c*num+d*ent
    ds.loc[ix,'combined']=combined_score

In [73]:
fire_final=ds.sort_values('combined',ascending=False).reset_index(drop=True)

In [65]:
fire_final.loc[16]

id                                         592251449054076928
text        RSS have sent 20000 swyamsevak to Nepal. Shiro...
labels                                                     -1
scores                                               0.998932
pnouns                                                      5
numbers                                                     2
ent                                                         6
combined                                             0.792415
Name: 16, dtype: object

In [71]:
max(0,-0.01)

0

In [74]:
txt=''
for ix in range(len(fire_final)):
    row=fire_final.loc[ix]
    sent=f'Nepal_Factual Q0 {row.id} {ix} {max(0.0,row.combined)} MIDAS_1\n'
    txt=txt+sent
file=open('../MIDAS_SEMI_AUTO.txt','w')
file.write(txt[:-1])
file.close()  

In [59]:
ds.loc[0].text

'RT @Economynext: #SriLanka to fly emergency medical help, food to #earthquake -struck #Nepal #lka #Economynext http://t.co/t6F2RXD4tj http:…'

In [75]:
fire_unsup=ds.copy()

In [77]:
fire_unsup['combined']=np.zeros((len(ds)))
for ix in range(len(ds)):
    #sc=fire_unsup.scores[ix]
    pn=fire_unsup.pnouns[ix]/(pnouns_max) 
    num=fire_unsup.numbers[ix]/(numbers_max)
    ent=fire_unsup.ent[ix]/ent_max
    b,c,d=0.33,0.33,0.33
    combined_score=b*pn+c*num+d*ent
    fire_unsup.loc[ix,'combined']=combined_score

In [81]:
fire_unsup_sort=fire_unsup.sort_values('combined',ascending=False).reset_index(drop=False)

In [83]:
txt=''
for ix in range(len(fire_final)):
    row=fire_unsup_sort.loc[ix]
    sent=f'Nepal_Factual Q0 {row.id} {ix} {max(0.0,row.combined)} MIDAS_2\n'
    txt=txt+sent
file=open('../MIDAS_UNSUPERVISED.txt','w')
file.write(txt[:-1])
file.close()  